<a href="https://colab.research.google.com/github/ethompson93/dmri_analysis_techniques/blob/main/ball_and_stick.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Compartment models: ball and stick**

In this coding assignment, we will be fitting a simple compartment model to the data: the ball and stick model.

Using the same phantom data as the previous exercise, download from https://drive.google.com/drive/folders/12hHKJoAXDB-AsNTzxXf4ZvSbfq-_7qmX?usp=share_link and upload it to this Google collab.

## Loading Variables

We'll re-use the code from the previous exercise to load in the data and extract the relevant variables.

In [ ]:
## Import libaries
import nibabel as nb
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import least_squares


In [ ]:
## Nifti Images

# Load our nifti image as "dwi_img" and get data to "dwi"
dwi_img = nb.load('fibrecup.nii.gz')
dwi = dwi_img.get_fdata()

# repeat this with the white matter mask
mask_img = nb.load('wm_mask.nii.gz')
mask = mask_img.get_fdata()

## Gradient information
# Load our gradient text file at grad
grad = np.loadtxt('grad.txt')

# select our gradient direction as "g"
g = grad[:,0:3]

# select our b-values as "b"
b = grad[:,3]

## Reshaping
# find the index of the voxels within the white matter mask
idx = np.where(mask>0)
print(np.shape(idx))

# extract dwi values at these WM voxels to produce a matrix of
# [WM voxels x grad-dirs]. Save this as "dwi_wm"
dwi_wm = dwi[idx[0],idx[1],idx[2]]

## Extracting b0 & non-b0 diffusion signals
# find from gradients where the b-value is equal to zero as "b0idx"
b0idx = np.where(b==0)[0]

# find from gradients where the b-value is NOT equal to zero as "non_b0idx"
non_b0idx = np.where(b!=0)[0]

# Extract baseline signal ("S0") by indexing "dwi" at b0idx
S0 = dwi_wm[:,b0idx]

# Extract gradient weighted signal ("S") by indexing the "dwi" at non-b0 values
S = dwi_wm[:,non_b0idx]

This produces four variables (shape of each variable in brackets) which we will need later on:

dwi - a 4D (64, 64, 3, 65) numpy array containing the value of diffusion at each voxel of the 3D image for each diffusion direction (65 directions).

mask - a 3D (64,64,3) binary array which provides a mask of the voxels corresponding to the brain. a value of 1 i given for each brain voxel, 0 for background.

g - gradient directions. A matrix (65,3) containing the direction vector for each diffusion direction.

b - corresponding b-values (65,) for each of these gradient directions (comment on whether this is single or multi-shell data)

# Visualising the data


In [ ]:
# make a figure with three subplots for b=0, b=2000 and the wm mask
fig, axs = plt.subplots(1, 3, figsize=(15, 5))
axs[0].imshow(dwi[:,:,1,0].T, origin="lower")
axs[0].set_title("b=0")
axs[1].imshow(np.mean(dwi[:,:,1,1:], axis=2).T, origin="lower")
axs[1].set_title("average b=2000")
axs[2].imshow(mask[:,:,1].T, origin="lower")
axs[2].set_title("white matter mask")
plt.show()

## Ball-and-stick model




The ball-and-stick model represents the dMRI signal as a weighted sum of contributions from an isotropic "ball" compartment, representing the free water in the voxel, and a fibre compartment (the "stick"):

$S_k=S(0)\left[(1- f) \exp (-b_k d)+ f \exp \left(-b_k d\left(\mathbf{g_k} \cdot \mathbf{v}\right)^2\right)\right]$

$S_k$ is the signal acquired with b-value $b_k$ and gradient direction $\mathbf{g_k}$. $f$ is the volume fraction of the fibre compartment, which has an orientation  described by unit vector $\mathbf{v}$. We assume the same diffusivity constant $d$ for both compartments.

**Coding Task:** create function "ball_and_stick" to generate the expected diffusion signal from the ball and stick model.

The function should return the voxel-wise signal for a set of acquisition parameters.

Use the following parameters as inputs :

*   angles describing the orientation of the stick: `theta` and `phi`
  (check out the [wikipedia page](https://en.wikipedia.org/wiki/Spherical_coordinate_system) on spherical coordinates for a refresher on how these relate to the x-y-z coordinates). These will be single float values for each voxel.
*   volume fraction of the stick component: `f`. A single float value for each voxel between 0 and 1.
*   diffusivity constant: `d`. We fix the value of `d` across all voxels and compartments.
*   baseline signal when b = 0: `S0`. Single value for the voxel (,1)
*   b-value of diffusion gradient: `b` (N,)
*   numpy array describing diffusion encoding gradients: `g` (N,3)

Your output should be a vector of size (N,) containing the simulated signal from the ball and stick model for each diffusion weighted direction.

N is the number of diffusion acquisitions, in this case 64.


In [ ]:
def ball(b, d):
  pass

def stick(b, g, d, theta, phi):
  pass

def ball_and_stick(theta, phi, f, d, S0, b, g):
  pass

In [ ]:
#@title ANSWERS

def ball(b, d):
  return np.exp(-b*d)

# long version
def stick(b, g, d, theta, phi):
  v = np.array([np.sin(theta)*np.cos(phi), np.sin(theta)*np.sin(phi), np.cos(theta)])
  N = len(b) # number of gradient directions
  result = np.zeros(N)  # Initialize result array
  # iterate through the gradients
  for i in range(N):
    result[i] = np.exp(-b[i] * d * (np.dot(v, g[i, :])**2))
  return result

# short version
def stick(b, g, d, theta, phi):
  v = np.array([np.sin(theta)*np.cos(phi), np.sin(theta)*np.sin(phi), np.cos(theta)])
  result = np.exp(-b * d * (np.dot(v, g.T)**2))
  return result


def ball_and_stick(theta, phi, f, d, S0, b, g):
  isotropic_signal = ball(b,d)
  anisotropic_signal = stick(b, g, d, theta, phi)
  return S0*((1-f)*isotropic_signal + f*anisotropic_signal)

Run the code below to check your function runs correctly, and outputs a vector of the expected size:

In [ ]:
vox_test = 0
d = 1.7e-3

result_test = ball_and_stick(theta=np.pi/2,
                             phi=np.pi,
                             f=0.5,
                             d=d,
                             S0 = S0[vox_test,:],
                             b=b[non_b0idx],
                             g=g[non_b0idx,:])

print(result_test)
print(result_test.shape)


Now we want to fit the model to our data. We have our acquisition parameters `b` and `g`,  and measured values for `S0`. We can also fix `d` based on information from the literature. The parameters we want to optimise are: `theta`, `phi` and `f`.

We'll be using scipy's non-linear least squares optimiser to fit the model. We have provided the code for this below. Take a look at the documentation to help you understand the inputs to the least_squares function: https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.least_squares.html. The first input we need to provide is a function which computes the vector of residuals, with the signature `fun(x, *args, **kwargs)`. Here, `x` is the variable being optimized. Since the minimization is performed with respect to `x`, we need to combine `theta`, `phi`, and `f` as a single NumPy array (ndarray) and pass it as `x`.

We have created a `residuals` function that calculates the difference between the measured signal `S` and the simulated signal from the `ball_and_stick` model:



In [ ]:
def residuals(x, S0, d, b, g, S):
  # x = (theta, phi, f)
  r = ball_and_stick(x[0], x[1], x[2], d, S0, b, g) - S
  return r

We pass this function to the `least_squares` optimiser, which uses it to minimise the sum-of-squared error between the model and the data. Take some time to look through the code below to understand the different parameters.

In [ ]:
d = 1.7e-3 # diffusion constant, found empirically
Nvox = np.shape(S0)[0]

# set up empty arrays to store the parameter estimates for each voxel
theta_estimates = np.zeros(Nvox)
phi_estimates = np.zeros(Nvox)
f_estimates = np.zeros(Nvox)

# we will also store the sum-of-squared residuals in each voxel, to evaluate the model fit
SSE = np.zeros(Nvox)

for i in range(Nvox):
  # we perform a least squares fit in each voxel
  res = least_squares(residuals,
              x0 = [np.pi/2, np.pi, 0.5], # initial guesses for theta, phi, f
              args=(S0[i,:], d, b[non_b0idx], g[non_b0idx,:], S[i,:]), # additional arguments to pass to the sum_squared_error function
              bounds =([0,0,0], [np.pi, 2*np.pi, 1]), # lower and upper bounds for theta, phi, f
              ftol = 1e-9, xtol=1e-9) # tolerance values for termination

  # We record voxel-wise estimates for theta, phi and f
  # from the output of the least_squares function
  theta, phi, f = res.x
  theta_estimates[i] = theta
  phi_estimates[i] = phi
  f_estimates[i] = f

  # we calculate the sum-of-squared errors from the residuals
  SSE[i] = np.sum(res.fun**2)

This can take a few minutes.

In the meantime, have a look at the extension tasks below.

## Visualisation
Once the least-squares fit has finished running, we can visualise our results.

First, we can look at the directions of the "stick" component. We convert the `theta` and `phi` estimates into regular Cartesian coordinates.



In [ ]:
# get the vectors in cartesian coordinates (x,y,z) for visualisation
dirs = np.array([np.sin(theta_estimates)*np.cos(phi_estimates),
                 np.sin(theta_estimates)*np.sin(phi_estimates),
                 np.cos(theta_estimates)])

dirs = dirs.T

We visualise the sticks in the same was as we did with the principal directions from DTI.

In [ ]:
# Choose a slice in which you will visualise the principal direction
slice_idx = np.where(idx[2]==1)

# Considering only the x and y components, plot a quiver plot of the prinicpal
# direction in each white matter voxel in this slice
# see matplotlib.pyplot.quiver documentation:
# https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.quiver.html

# x and y correspond to location of each arrow,
# i.e. the coordinates of the white matter voxel
x = idx[0][slice_idx]
y = idx[1][slice_idx]

# u and v correspond to the x and y components of the principal direction in
# each voxel
u = dirs[slice_idx,0]
v = dirs[slice_idx,1]

fig = plt.figure
plt.quiver(x,y,u,v, scale_units='x', scale=1)

plt.show()

Compare this output to the one from the previous exercise. Can you see any differences?

Next, we can plot the volume fractions of the stick components, `f`.

In [ ]:
# plot the fiber volume fractions
f_map = np.zeros_like(mask)
f_map[idx[0],idx[1],idx[2]] = f_estimates

plt.imshow(f_map[:,:,1].T, origin="lower")
plt.colorbar()
plt.show()


We can also look at the error between the model fit and the measured signal. We can this to compare these results with the models from the other exercises.

In [ ]:
SSE_map = np.zeros_like(mask)
SSE_map[idx[0],idx[1],idx[2]] = SSE

plt.imshow(SSE_map[:,:,1].T, origin="lower", vmax = 5000)
plt.colorbar()
plt.show()

print(f"median sum-of-squared error = {np.median(SSE)}")

What do you notice about the areas with high error in the model fit?

# Extension tasks:

*   What are the pros and cons of the ball and stick model?
*   Can you adjust the parameters in the least_squares optimiser to improve the model fit?
*   What other compartments could we add to improve the accuracy of the model? Can you implement them?
*   Look at the different methods available in scipy.optimise. How would you use these to fit the parameters? (https://docs.scipy.org/doc/scipy/reference/optimize.html).

